## Dhairya Kothari
## PEX Machine Learning challenge

### Part 1 - Capture frames from a video

In this step, we download a youtube video in .mp4 format and extract frames from it. Alternatively we can also do web scraping using curl or a crawler and get frames from the video. I chose to download the YouTube video as the internet connection was quite unreliable at my place (spartans were less on amenities, eh! =D ), so I rather worked with offline data stored in my system. I downloaded the YouTube video using a freeware available online. <br>
Keeping in mind all the DMCA requirements at the time of download and Also giving a fair use disclaimer that all rights belong to the respective creators and this exercise was only meant for educational/testing purposes. No money was made using the video. <br> 
I will upload the images to the github repo, not the full youtube videos. I can provide the link to the said youtube videos though. 

#### Snapshot of the directory

This is how the directory structure will look after the end of the code: <br>
<p>
    ./data <br>
    &nbsp;&nbsp;&nbsp;&nbsp; /train_data <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; /indoor <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; /........ images ....... <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; /outdoor <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; /........ images ....... <br>
<p>
    &nbsp;&nbsp;&nbsp;&nbsp; /val_data <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; /indoor <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; /........ images ....... <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; /outdoor <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; /........ images ....... <br>    

In [1]:
import cv2
import math
import glob
import shutil
import os, random

import numpy as np
from PIL import Image

We use OpenCV(cv2) to capture frame from a video. We have the freedom to choose and save which frames we want, because it can go through each and every frame. Considering that YouTube video(s) are well crafted, dont have too many jumpcuts and are generally decodable  when you freeze a frame (i.e. not all frames are blurry when you freeze them). Also keeping in mind that the video(s) are consistent and within 3-5 seconds time window in a video we have the same image/information. <br>
Here we store the FPS and develop a mechanism to capture one frame each second (to reduce redundency) and then introduce a programmable delay (for example 5 seconds) to further refine the data set. Then we have one frame every 5 seconds, as previously when I captured one frame per second, the data set had a lot of redundent images. <br>
<p>
   Keep in mind that encapsulating the whole code into a function and/or making this python file into a package is completely doable in 5 mins work, here I have not done this because I do not want to restrict the code to the format of my defined function, but one can do it based on what and how he wants the code to work with their current pipeline. 
</p>

#### Indoor images from video

In [5]:
# Playing video from file:
indoor = cv2.VideoCapture('indoor.mp4')

try:
    if not os.path.exists('data/train_data/indoor'):
        os.makedirs('data/train_data/indoor')
except OSError:
    print ('Error: Creating directory of data')

fps = indoor.get(5) #get framerate
delay = 4 #introduce a delay in seconds
currentFrame = 1
while(True):
    f_no = indoor.get(1) #current frame number
    # Capture frame-by-frame
    ret, frame = indoor.read()
    
    if not ret: break

    if (f_no % math.floor(fps*delay) == 0):
        # Saves image of the current frame in jpg file
        name = './data/train_data/indoor/' + str(currentFrame) + '.jpg'
        cv2.imwrite(name, frame)
        currentFrame += 1

# release the capture and destroy all windows
indoor.release()
cv2.destroyAllWindows()

#### Outdoor images from video

In [4]:
outdoor = cv2.VideoCapture('outdoor.mp4')
outdoor1 = cv2.VideoCapture('outdoor1.mp4')

try:
    if not os.path.exists('data/train_data/outdoor'):
        os.makedirs('data/train_data/outdoor')
except OSError:
    print ('Error: Creating directory of data')

currentFrame = 1
fps = outdoor.get(5)
delay = 3
while(True):
    f_no = outdoor.get(1)
    ret, frame = outdoor.read()
    
    if not ret: break

    if (f_no % math.floor(fps*delay) == 0):
        name = './data/train_data/outdoor/' + str(currentFrame) + '.jpg'
        cv2.imwrite(name, frame)
        currentFrame += 1

fps1 = outdoor1.get(5)
delay = 10
# here is an example of grabing frames from 2 videos, we just continue the currentframe number from where it left off previously
while(True):
    f_no = outdoor1.get(1)
    ret, frame = outdoor1.read()
    
    if not ret: break

    if (f_no % math.floor(fps1*delay) == 0):
        name = './data/train_data/outdoor/' + str(currentFrame) + '.jpg'
        cv2.imwrite(name, frame)
        currentFrame += 1        

outdoor.release()
outdoor1.release()
cv2.destroyAllWindows()

#### Creating a validation set
We select 20% of random images from indoor and outdoor directories and move them to respective validation set creating an 80-20 split 

In [6]:
tr_sample_N_in = len(glob.glob('./data/train_data/indoor/*.jpg'))
tr_sample_N_out = len(glob.glob('./data/train_data/outdoor/*.jpg'))

try:
    if not os.path.exists('data/val_data/indoor'):
        os.makedirs('data/val_data/indoor')
except OSError:
    print ('Error: Creating directory of data')

try:
    if not os.path.exists('data/val_data/outdoor'):
        os.makedirs('data/val_data/outdoor')
except OSError:
    print ('Error: Creating directory of data')


for i in range(0,int(0.2*tr_sample_N_in)):
    rand_img = random.choice(os.listdir("./data/train_data/indoor/"))
    #print(rand_img)
    shutil.move("./data/train_data/indoor/"+rand_img, "./data/val_data/indoor/"+rand_img)

for i in range(0,int(0.2*tr_sample_N_out)):
    rand_img = random.choice(os.listdir("./data/train_data/outdoor/"))
    #print(rand_img)
    shutil.move("./data/train_data/outdoor/"+rand_img, "./data/val_data/outdoor/"+rand_img)

#### Part two saved in another notebook named: <i>In_Out </i>